# Cuisine segmentation 

In [ ]:
cuisine_df = pd.read_excel("data/cusine.xlsx")

In [ ]:
aisles_df = pd.read_csv("data/aisles.csv")
departments_df = pd.read_csv("data/departments.csv")
order_products__prior_df = pd.read_csv("data/order_products__prior.csv")
order_products__train_df = pd.read_csv("data/order_products__train.csv")
orders_df = pd.read_csv("data/orders.csv")
products_df = pd.read_csv("data/products.csv")
sample_submission_df = pd.read_csv("data/sample_submission.csv")

In [ ]:
products_df = products_df[products_df["aisle_id"].isin([2,4,5,7,9,12,13,14,16,21,30,38,42,59,63,65,66,67,68,76,79,81,104,110,122,128,131])]

In [ ]:
req_prod_ids = products_df["product_id"]

In [ ]:
order_products__prior_df = order_products__prior_df[order_products__prior_df["product_id"].isin(req_prod_ids)]

In [ ]:
order_products__prior_df = order_products__prior_df.merge(products_df[["product_id", "product_name"]],on="product_id",how="inner")

In [ ]:
prior_order_prods = order_products__prior_df.groupby('order_id')['product_name'].apply(list).to_frame().reset_index()

In [ ]:
prior_order_df = prior_order_prods.merge(orders_df,on="order_id",how="inner")

In [ ]:
prior_orders = prior_order_df.groupby('user_id')['product_name'].apply(list).to_frame().reset_index()
prior_orders = prior_orders.rename(columns={"product_name": "orders"})
cuisine_df = cuisine_df.groupby('cuisine')['ingredient'].apply(list).transform(lambda x: ','.join(x)).reset_index()
cuisine_df = [x+": "+y for x,y in zip(cuisine_df["cuisine"],cuisine_df["ingredient"])]

# Assymetric semantic search

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
sbert_model = SentenceTransformer("sentence-transformers/msmarco-distilbert-cos-v5")

In [ ]:
cuisine_embeddings = sbert_model.encode(cuisine_df, convert_to_tensor=True)

In [ ]:
import re

In [ ]:
for i in range(0,5):
    for order in prior_orders["orders"][i]:
        for product in order:
            
            order_embedding = sbert_model.encode(re.sub(r'[^A-Za-z ]+', '', product), convert_to_tensor=True)
            scores = util.cos_sim(order_embedding, cuisine_embeddings)[0]
            matched_cuisines = torch.topk(scores, k=1)

            print("\n\n======================\n\n")
            print("Query:", re.sub(r'[^A-Za-z ]+', '', product))
            print("\nSimilar cuisines:")

            for score, idx in zip(matched_cuisines[0], matched_cuisines[1]):
                print(cuisine_df[idx], "(Score: {:.4f})".format(score))

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2", max_length=512)

In [ ]:
product_cuisine_map = []
for product in tqdm(list(products_df["product_name"])):
    scores = model.predict([(re.sub(r'[^A-Za-z ]+', '', product), cuisine) for cuisine in cuisine_df_3])
    cuisine = np.argmax(scores)
    product_cuisine_map.append(cuisine)

In [ ]:
products_df["cuisine"] = product_cuisine_map

In [ ]:
products_df.to_csv("product_cusine_df.csv")

In [ ]:
cuisines = [i[:i.find(":")] for i in cuisine_df_3]

In [ ]:
products_df["cuisine"] = products_df["cuisine"].apply(lambda x:cuisines[x])

In [ ]:
order_products__prior_df = pd.read_csv("data/order_products__prior.csv")
orders_df = pd.read_csv("data/orders.csv")

In [ ]:
req_prod_ids = products_df["product_id"]

In [ ]:
order_products__prior_df = order_products__prior_df[order_products__prior_df["product_id"].isin(req_prod_ids)]
order_products__prior_df = order_products__prior_df.merge(products_df[["product_id", "cuisine"]],on="product_id",how="inner")

In [ ]:
cuisine_orders = order_products__prior_df["cuisine"].value_counts().to_frame().reset_index()

In [ ]:
labels = [Patch(facecolor="#FFFFFF",label = str(x)+" - "+str(y)) for x,y in zip(cuisine_orders["index"],cuisine_orders["cuisine"])]

In [ ]:
from matplotlib.patches import Patch

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Order count for various cuisine related products")
plt.axis('off')
squarify.plot(sizes=cuisine_orders["cuisine"], label=cuisine_orders["index"], alpha=.7,pad=2)
plt.legend(handles=labels,loc='center right',bbox_to_anchor=(1.4, 0.5),frameon=False)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import squarify

In [ ]:
prior_order_cuisines = order_products__prior_df.groupby('order_id')['cuisine'].apply(list).to_frame().reset_index()